In [17]:
"""
Project 2: Data Preprocessing & Association Rule Mining
Google Colab Version
"""

import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import warnings
warnings.filterwarnings('ignore')

!pip install mlxtend

from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]
print(f"File uploaded: {filename}")

print("\n" + "="*60)
print("LOADING DATA")
print("="*60)

df_original = pd.read_csv(filename)
print(f"Shape: {df_original.shape}")
print(f"\nFirst 10 rows:")
print(df_original.head(10))

print(f"\nColumn names: {list(df_original.columns)}")
print(f"\nData types:\n{df_original.dtypes}")


print("\n" + "="*60)
print("DATA QUALITY ASSESSMENT")
print("="*60)

print("\n1. Missing Values:")
missing = df_original.isnull().sum()
print(missing)
print(f"Total missing values: {missing.sum()}")

print("\n2. Duplicate Transactions:")
if 'Transaction_ID' in df_original.columns:
    duplicates = df_original.duplicated(subset=['Transaction_ID']).sum()
    print(f"Duplicate Transaction IDs: {duplicates}")
    if duplicates > 0:
        print("Duplicate rows:")
        print(df_original[df_original.duplicated(subset=['Transaction_ID'], keep=False)])

print("\n3. Negative Values:")
if 'Quantity' in df_original.columns:
    qty_numeric = pd.to_numeric(df_original['Quantity'], errors='coerce')
    neg_qty = (qty_numeric < 0).sum()
    print(f"Negative Quantities: {neg_qty}")
    if neg_qty > 0:
        print(df_original[qty_numeric < 0][['Transaction_ID', 'Product_Name', 'Quantity']])

if 'Price' in df_original.columns:
    price_numeric = pd.to_numeric(df_original['Price'], errors='coerce')
    neg_price = (price_numeric < 0).sum()
    print(f"Negative Prices: {neg_price}")
    if neg_price > 0:
        print(df_original[price_numeric < 0][['Transaction_ID', 'Product_Name', 'Price']])

print("\n4. Format Inconsistencies:")
if 'Payment_Method' in df_original.columns:
    print(f"Payment Methods: {df_original['Payment_Method'].unique()}")
    print(f"Value counts:\n{df_original['Payment_Method'].value_counts()}")

if 'Transaction_Status' in df_original.columns:
    print(f"\nTransaction Statuses: {df_original['Transaction_Status'].unique()}")
    print(f"Value counts:\n{df_original['Transaction_Status'].value_counts()}")

print("\n5. Date Issues:")
if 'Transaction_Date' in df_original.columns:
    print(f"Sample dates: {df_original['Transaction_Date'].head(10).tolist()}")
    try:
        valid_dates = pd.to_datetime(df_original['Transaction_Date'], errors='coerce')
        invalid = valid_dates.isnull().sum()
        print(f"Invalid/unparseable dates: {invalid}")
    except:
        print("Error parsing dates")

print("\n" + "="*60)
print("DATA CLEANING")
print("="*60)

df_clean = df_original.copy()
df_clean = df_clean.dropna(subset=['Transaction_ID'])

df_clean = df_clean.drop_duplicates(subset=['Transaction_ID'], keep='first')

if 'Quantity' in df_clean.columns:
    df_clean['Quantity'] = pd.to_numeric(df_clean['Quantity'], errors='coerce')
    df_clean['Quantity'] = df_clean['Quantity'].abs()
    df_clean['Quantity'].fillna(1, inplace=True)

if 'Price' in df_clean.columns:
    df_clean['Price'] = pd.to_numeric(df_clean['Price'], errors='coerce')
    median_price = df_clean['Price'].median()
    df_clean['Price'].fillna(median_price, inplace=True)
    df_clean['Price'] = df_clean['Price'].abs()

if 'Payment_Method' in df_clean.columns:
    df_clean['Payment_Method'] = df_clean['Payment_Method'].str.lower().str.replace(' ', '')

if 'Transaction_Status' in df_clean.columns:
    df_clean['Transaction_Status'] = df_clean['Transaction_Status'].fillna('Unknown')
    df_clean['Transaction_Status'] = df_clean['Transaction_Status'].str.lower().str.capitalize()
    df_clean['Transaction_Status'] = df_clean['Transaction_Status'].replace({'Complete': 'Completed'})

if 'Transaction_Date' in df_clean.columns:
    df_clean['Transaction_Date'] = pd.to_datetime(df_clean['Transaction_Date'], errors='coerce')
    df_clean['Transaction_Date'].fillna(pd.Timestamp('2023-01-01'), inplace=True)

if 'Product_Name' in df_clean.columns:
    df_clean['Product_Name'] = df_clean['Product_Name'].str.strip()
    df_clean['Product_Name'] = df_clean['Product_Name'].replace({'Tab': 'Tablet', 'tab': 'Tablet'})

print("\n" + "="*60)
print("CLEANING SUMMARY")
print("="*60)
print(f"Original rows: {len(df_original)}")
print(f"Cleaned rows: {len(df_clean)}")
print(f"Rows removed: {len(df_original) - len(df_clean)}")

transactions = df_clean.groupby('Customer_ID')['Product_Name'].apply(list).values.tolist()

# Transform to one-hot encoding
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

min_support = 0.1
min_confidence = 0.3

frequent_itemsets = apriori(df_encoded, min_support=min_support, use_colnames=True)

rules_display = pd.DataFrame()
if len(frequent_itemsets) > 0:
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)
    if len(rules) > 0:
        rules = rules.sort_values('lift', ascending=False)
        rules_display = pd.DataFrame({
            'Antecedent': [', '.join(list(x)) for x in rules['antecedents']],
            'Consequent': [', '.join(list(x)) for x in rules['consequents']],
            'Support (%)': (rules['support'] * 100).round(1),
            'Confidence (%)': (rules['confidence'] * 100).round(1),
            'Lift': rules['lift'].round(2)
        })

transactions_path = '/content/transactions_cleaned.csv'
rules_path = '/content/association_rules.csv'

df_clean.to_csv(transactions_path, index=False)
rules_display.to_csv(rules_path, index=False)

print(f"\n✓ Files saved successfully:")
print(f" - {transactions_path}")
print(f" - {rules_path}")
print("\nGo to the left File Explorer in Colab to download these files manually.")


Saving dirty_financial_transactions.csv to dirty_financial_transactions (2).csv
File uploaded: dirty_financial_transactions (2).csv

LOADING DATA
Shape: (100000, 8)

First 10 rows:
  Transaction_ID Transaction_Date Customer_ID    Product_Name  Quantity  \
0          T0001       2024-08-02       C2205      Headphones      -5.0   
1          T0002       2020-02-10       C3156         Coffee      469.0   
2          T0003       2025-02-30       C2919          Tablet      -4.0   
3          T0004       2020-08-17       C3009             Tab      -7.0   
4          T0005       2025-02-30       C3488  Coffee Machine     -10.0   
5          T0006       2021-10-26       C4241      Smartphone     598.0   
6            NaN       2025-02-30       C1313          Laptop      10.0   
7          T0008       2023-13-01       C4736      Headphones     669.0   
8          T0009              NaN       C3387          Tablet      10.0   
9          T0010       2025-02-30       C2846          Laptop      -1